In [251]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
np.random.seed(22)
from cvxopt import matrix, solvers


In [252]:
class Kernel_SVM():

    def __init__(self, C, dataframe, gamma = None):
        self.C  = C
        self.X = dataframe[:,:-1].T #matrix
        self.y = dataframe[:, -1]  # 1d array
        self.num_sample = dataframe.shape[0]  # # of row
        self.num_feature = dataframe.shape[1] - 1  #remove label column
        self.gamma = gamma


    def G_Kernel(self,x1,x2, gamma):
        'x1, x2: collumn vector'
        return np.exp(-gamma*np.sum((x1 - x2)**2))    
    
    def fit(self):
        #check gamma is not none:
        if not self.gamma:
            self.gamma = 1/self.num_sample


        #create kernel matrix
        kernel_matrix = np.zeros( (self.num_sample, self.num_sample))
        
        for column in range(self.num_sample):
            for row in range(self.num_sample):
                kernel_matrix[column, row] = self.y[column]*self.y[row]*self.G_Kernel(self.X[:,column],self.X[:, row], self.gamma)
        
        #Optimization part:

        P = matrix(kernel_matrix, tc= 'd')
        q = matrix( -np.ones((self.num_sample, 1)), tc ='d')

        G1 = ( - np.eye(self.num_sample))
        G2 = (np.eye(self.num_sample))
        G = np.concatenate((G1,G2), axis = 0) 
        G = matrix(G, tc ='d')

        h1 = np.zeros((self.num_sample,1))  #lambda >= 0 
        h2 = self.C*np.ones((self.num_sample,1)) #lambda < C 
        h = np.concatenate((h1, h2), axis = 0)
        h = matrix( h, tc = 'd')

        A = matrix(np.array([self.y]), tc = 'd')
        b = matrix(np.zeros((1, 1)), tc='d')

        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)

        pre_lambda = np.array(sol['x']) #2d array
        # print(pre_lambda)
        # TAKING SUBSET S: S = {lambda: lambda > 0}
        positive_index = np.where(pre_lambda > 1e-6)

        #positive_index[0]: location of corresponding n_th sample
        self.lambda_subset_S = pre_lambda[positive_index[0],:]  #column vec
        self.y_subset_S = np.array([self.y])[:, positive_index[0]].T  #column vec
        self.X_subset_S = self.X[:, positive_index[0]]   #matrix


        # TAKING SUBSET M: M = {lambda: 0<lambda<C}
        M_index = np.where((pre_lambda < self.C) & (pre_lambda > 1e-6))
        self.lambda_subset_M = pre_lambda[M_index[0],:]  #column vec
        self.y_subset_M = np.array([self.y])[:, M_index[0]].T  #column vec
        self.X_subset_M = self.X[:,  M_index[0]]   #matrix
 
        
    def kernel_sum(self, lambda_vec, y, X, input):   
    #sum( lambda * y * kernel(X,input) )  
    #X: matrix, input: col_vector => kernel(X,input) : col_vector
    #all input is collum vec
        kernel_product = np.array([ self.G_Kernel(X[:,i], input, self.gamma ) for i in range(X.shape[1])  ]).reshape(-1,1) 
        return np.sum(lambda_vec*y*kernel_product)

    def prediction(self, input):
        #reshape_input to column vec
        if input.shape[1] != 1:
            input = input.reshape(-1,1)

        #loop through subset M:
        sum_M = 0
        for i in range(self.lambda_subset_M.shape[0]):
            sum_M += self.y_subset_M[i,:] -  self.kernel_sum(self.lambda_subset_S, self.y_subset_S, self.X_subset_S, self.X_subset_M[:,i])
        
        #sum subset S:
        sum_S = self.kernel_sum(self.lambda_subset_S, self.y_subset_S, self.X_subset_S, input)

        prediction = sum_S + (1/self.lambda_subset_M.shape[0])*sum_M

        #print(prediction)
        return np.sign(prediction)
    
            


In [253]:

means = [[2, 2], [4, 2]]
cov = [[.3, .2], [.2, .3]]
N = 100
X0 = np.random.multivariate_normal(means[0], cov, N) # class 1
X1 = np.random.multivariate_normal(means[1], cov, N) # class 2



class_1 = np.concatenate( (X0, np.ones(  (X0.shape[0],1)  ) ), axis = 1)
#print(class_1)#
class_2 = np.concatenate( (X1,- np.ones(  (X0.shape[0],1)  ) ), axis = 1)
#print(class_2)
sample = np.concatenate((class_1, class_2), axis = 0)
#print(sample)

In [254]:
SVM_v2 = Kernel_SVM(C=1, dataframe = sample)
SVM_v2.fit()

In [255]:
prediction = SVM_v2.prediction(np.array([[1,1]]))
prediction

array([1.])